# Sentiment analysis for Sentiment140 with CANINE

## Install HuggingFace dependencies

In [ ]:
!pip install -q datasets
!pip install -q transformers

     |████████████████████████████████| 325 kB 30.5 MB/s 
     |████████████████████████████████| 67 kB 7.1 MB/s 
     |████████████████████████████████| 1.1 MB 63.0 MB/s 
     |████████████████████████████████| 136 kB 73.2 MB/s 
     |████████████████████████████████| 212 kB 70.0 MB/s 
     |████████████████████████████████| 127 kB 71.6 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 144 kB 73.6 MB/s 
     |████████████████████████████████| 271 kB 68.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 3.8 MB 17.2 MB/s 
     |████████████████████████████████| 6.5 MB 57.3 MB/s 
     |████████████████████████████████| 596 kB 66.1 MB/s 
     |████████████████████████████████| 8

## Create functions to download and preproccess data

In [ ]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from datasets import load_dataset
from transformers import CanineTokenizer, DataCollatorWithPadding

bz = 64  # batch size

tokenizer = CanineTokenizer.from_pretrained("google/canine-s")

def encode(samples):
    return tokenizer(samples['text'], padding=False, truncation=True)

def quantify(samples):
    return {'label': np.digitize(samples['sentiment'], np.array([2]), right=True)}

letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
           'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
def typo(samples, typo_percent=0):
    for i,sample in enumerate(samples['text']):
        for j,x in enumerate(sample):
            if np.random.random() < typo_percent and x.isalpha():
                aux = list(samples['text'][i])
                aux[j] = np.random.choice(letters)
                samples['text'][i] = ''.join(aux)
    return samples

def generate_dataset(typo_percent=0):
    dataset = load_dataset('sentiment140')

    # Remove neutral ids
    non_neutral_ids = np.where(np.array(dataset['test']['sentiment']) != 2)[0]
    dataset['test'] = dataset['test'].select(non_neutral_ids)

    # Split train in train and valid
    dataset_aux = dataset['train'].train_test_split(0.01, seed=0)
    dataset['train'] = dataset_aux['train']
    dataset['validation'] = dataset_aux['test']

    if typo_percent > 0:
        dataset = dataset.map(typo, batched=True, batch_size=bz, fn_kwargs={'typo_percent':typo_percent})
    dataset = dataset.map(encode, batched=True, batch_size=bz)
    dataset = dataset.map(quantify, batched=True, batch_size=bz)
    
    dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

    loader = {}
    for split in dataset:
        loader[split] = DataLoader(dataset[split], batch_size=bz, shuffle=(split=='train'), collate_fn=DataCollatorWithPadding(tokenizer))
    
    return dataset, loader

dataset, loader = generate_dataset(typo_percent=0)

Downloading:   0%|          | 0.00/657 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892 [00:00<?, ?B/s]

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


Generating train split:   0%|          | 0/1600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/498 [00:00<?, ? examples/s]

...

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/250 [00:00<?, ?ba/s]

  0%|          | 0/24750 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/250 [00:00<?, ?ba/s]

## Create train and eval functions

In [ ]:
from tqdm import tqdm
import copy
import math

def train_canine(net, device, optimizer, criterion, lr, lr_scheduler, train_loader, val_loader, epochs, unfreeze=False, unfreeze_ep=5):
    best_loss = np.Inf
    best_ep = 1
    iters = []
    train_losses = []
    val_losses = []
    log_interval = math.ceil(len(train_loader)/20)+1

    for ep in range(epochs):
        with tqdm(total=len(train_loader), unit_scale=True, postfix={'Loss':0.0, 'Accuracy':0.0, 'lr':lr_scheduler.get_last_lr()[0]},
                        desc="Epoch : %i/%i" % (ep+1, epochs)) as pbar:
            net.train()
            total_loss = 0.0
            correct = 0
            nb_samples = 0
            for it, batch in enumerate(train_loader):
                # Converting to cuda tensors
                batch = {k: v.to(device) for k, v in batch.items()}
                seq, attn_masks, token_type_ids, labels = \
                    batch['input_ids'], batch['attention_mask'], batch['token_type_ids'], batch['labels']
                labels = labels.to(torch.int64)
        
                # Clear gradients
                optimizer.zero_grad()

                # Forward pass
                output = net(input_ids=seq, attention_mask=attn_masks,
                             token_type_ids=token_type_ids, labels=labels)

                # Backpropagating the gradients
                loss = output.loss #HUGGINGFACE
                #loss = criterion(output, labels)
                loss.backward()
                # Optimization step
                optimizer.step()

                total_loss += loss.item()

                logits = output.logits #HUGGINGFACE
                #logits = output
                predicted = logits.argmax(axis=1)
                correct += predicted.eq(labels).sum().item()
                nb_samples += len(predicted)

                pbar.set_postfix({'Loss':total_loss/(it%log_interval+1), 'Accuracy':100*correct/nb_samples, 'lr':lr_scheduler.get_last_lr()[0]})
                pbar.update(1)
                
                lr_scheduler.step()

                if it % log_interval == log_interval-1:
                    total_loss = 0.0
                    correct = 0
                    nb_samples = 0
                    val_loss, val_acc = evaluate_loss(net, device, val_loader, criterion)  # Compute validation loss
                    print(f"Val Loss : {val_loss:.3f}. Val Accuracy {val_acc:.2f}%")

                    if val_loss < best_loss:
                        print(f"Best validation loss improved from {best_loss:.3f} to {val_loss:.3f}")
                        print()
                        net_copy = copy.deepcopy(net)  # save a copy of the model
                        best_loss = val_loss
                        best_ep = ep + 1
                        path_to_model=f'model.pt'
                        torch.save(net_copy.state_dict(), path_to_model)

        val_loss, val_acc = evaluate_loss(net, device, val_loader, criterion)  # Compute validation loss
        print()
        print(f"Epoch {ep+1} complete! Val Loss : {val_loss:.3f}. Val Accuracy {val_acc:.2f}%")

        train_losses.append(total_loss/len(train_loader))
        val_losses.append(val_loss)

        if val_loss < best_loss:
            print(f"Best validation loss improved from {best_loss:.3f} to {val_loss:.3f}")
            print()
            net_copy = copy.deepcopy(net)  # save a copy of the model
            best_loss = val_loss
            best_ep = ep + 1
        
        if unfreeze and ep == unfreeze_ep:
            for param in net.parameters():
                param.requires_grad = True

    # Saving the model
    path_to_model=f'model.pt'
    torch.save(net_copy.state_dict(), path_to_model)
    print(f"The model has been saved in {path_to_model}")

    del loss
    torch.cuda.empty_cache()

    return train_losses, val_losses, net_copy

In [ ]:
from tqdm import tqdm

def evaluate_loss(net, device, dataloader, criterion, disable_tqdm=True):
    net.eval()

    total_loss = 0
    correct = 0

    print('Evaluating...')
    with torch.no_grad():
        for it, batch in enumerate(tqdm(dataloader, disable=disable_tqdm)):
            batch = {k: v.to(device) for k, v in batch.items()}
            seq, attn_masks, token_type_ids, labels = \
                batch['input_ids'], batch['attention_mask'], batch['token_type_ids'], batch['labels']
            labels = labels.to(torch.int64)
            output = net(seq, attn_masks, token_type_ids, labels=labels)
            total_loss += output.loss.item() #HUGGINGFACE
            #total_loss += criterion(output, labels).item()
            predicted = output.logits.argmax(axis=1) #HUGGINGFACE
            #predicted = output.argmax(axis=1)
            correct += predicted.eq(labels).sum().item()

    return total_loss/len(dataloader), 100*correct/len(dataloader.dataset)

## Train the model

In [ ]:
print("Creation of the models' folder...")
!mkdir models

Creation of the models' folder...


In [ ]:
#freeze_canine = False  # if True, freeze the encoder weights and only update the classification layer weights
#maxlen = 2048  # maximum length of the tokenized input pair : if greater than "maxlen", the input is truncated and else if smaller, the input is padded
lr = 2e-5  # learning rate
epochs = 2  # number of training epochs

In [ ]:
from transformers import CanineTokenizer, CanineConfig, CanineForSequenceClassification, get_linear_schedule_with_warmup, DistilBertForSequenceClassification#, AdamW
from torch.optim import AdamW

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
losses = []
accs = []

typo_percent = 0

configuration = CanineConfig(num_labels=2)
net = CanineForSequenceClassification.from_pretrained("google/canine-s", config=configuration)

net.to(device)

optimizer = AdamW(net.parameters(), lr=lr, weight_decay=1e-2)
criterion = nn.CrossEntropyLoss()
t_total = len(loader['train']) * epochs  # Necessary to take into account Gradient accumulation
num_warmup_steps = int(t_total*0.1) # The number of steps for the warmup phase.
lr_scheduler = get_linear_schedule_with_warmup(optimizer=optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=t_total)

train_losses, val_losses, net = train_canine(net, device, optimizer, criterion, lr, lr_scheduler, loader['train'], loader['validation'], epochs)

print(f'\n### Typo percentage = {typo_percent} ###')
losses_aux = []
accs_aux = []
for split in ['train', 'validation', 'test']:
    loss, acc = evaluate_loss(net, device, loader[split], criterion, disable_tqdm=False)
    losses_aux.append(loss)
    accs_aux.append(acc)
    print()
    print(split)
    print(f'   loss = {loss}\n   acc = {acc}')

losses.append(losses_aux)
accs.append(accs_aux)

Downloading:   0%|          | 0.00/504M [00:00<?, ?B/s]

Some weights of CanineForSequenceClassification were not initialized from the model checkpoint at google/canine-s and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch : 1/2:   5%|▌         | 1.24k/24.8k [13:52<4:24:37, 1.48it/s, Loss=0.668, Accuracy=57.2, lr=5e-6]

Evaluating...
Val Loss : 0.565. Val Accuracy 70.84%
Best validation loss improved from inf to 0.565



Epoch : 1/2:  10%|█         | 2.48k/24.8k [29:02<4:09:53, 1.49it/s, Loss=0.491, Accuracy=76.3, lr=1e-5]

Evaluating...
Val Loss : 0.455. Val Accuracy 78.60%
Best validation loss improved from 0.565 to 0.455



...

Epoch : 2/2:  60%|██████    | 14.9k/24.8k [3:01:46<1:53:28, 1.45it/s, Loss=0.249, Accuracy=89.7, lr=4.44e-6]

Evaluating...
Val Loss : 0.299. Val Accuracy 87.22%


Epoch : 2/2:  65%|██████▌   | 16.1k/24.8k [3:16:57<1:37:35, 1.48it/s, Loss=0.247, Accuracy=89.9, lr=3.88e-6]

Evaluating...
Val Loss : 0.302. Val Accuracy 87.27%


Epoch : 2/2:  66%|██████▌   | 16.3k/24.8k [3:20:46<1:35:17, 1.47it/s, Loss=0.243, Accuracy=90.2, lr=3.78e-6]

## Evaluate model

In [ ]:
from transformers import CanineConfig, CanineForSequenceClassification

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

configuration = CanineConfig(num_labels=2)
net = CanineForSequenceClassification.from_pretrained("google/canine-s", config=configuration)
net.to(device)

path_to_model='model.pt'
net.load_state_dict(torch.load(path_to_model))

criterion = nn.CrossEntropyLoss()

losses = []
accs = []
for split in ['train', 'validation', 'test']:
    loss, acc = evaluate_loss(net, device, loader[split], criterion, disable_tqdm=False)
    losses.append(loss)
    accs.append(acc)
    print()
    print(split)
    print(f'   loss = {loss}\n   acc = {acc}')

Some weights of CanineForSequenceClassification were not initialized from the model checkpoint at google/canine-s and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluating...


100%|██████████| 6/6 [00:00<00:00,  6.63it/s]


test
   loss = 0.3904445419708888
   acc = 82.45125348189416
